# Wall Thickness

In [ ]:
from pathlib import Path
# from project_heart.lv import LV
from project_heart.utils import set_jupyter_backend
from project_heart.enums import *
set_jupyter_backend("pythreejs")
import logging
import numpy as np
float_formatter = "{:.5f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

from project_heart.examples import get_lv_ideal
from project_heart.examples import get_lv_typeA

lv_ideal = get_lv_ideal(Path("../../_static/sample_files/ideal_linear_pressure_increase copy.xplt"))
lv_typeA = get_lv_typeA(
    Path("../../_static/sample_files/lv_typeA_hex.vtk"),
    Path("../../_static/sample_files/sample_displacement_lv_typeA_hex_states_linear_press_incr.pbz2")
    )

sample_spk_typeA_endo = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="endo", spk_collection="SAMPLE")[0]
sample_spk_typeA_epi = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="epi", spk_collection="SAMPLE")[0]

## Computing thickness

**Definition**: Distance between Epicardium and Endocardium.

The distance between the Epicardium and Endocardium regions is used to calculate wall thickness. Due to the possibility that the quantity of speckles in both regions may differ and not reflect the same relative location, which prevents us from measuring the euclidean distance between the two speckles, we employ [radial metrics](./radial_metrics.ipynb). As a result, thickness is calculated as the simple difference between a radial metric at the epicardium and the endocardium. 

Our implementation consider the usage of [Speckles](../basic_definitions/speckles.ipynb), which helps to approximate a clinical setting, facilitates the computation for certain metrics and helps to minimizing errors due to noise in geometry by averaging values based on local regions. See [docs](../basic_definitions/speckles.ipynb) for details. Fo wall thickness, we apply validation checks to confirm speckles at Endocardium and Epicardium are related (close reference center at longitudinal axis and similar subsets).

The final metric is computed by applying [reduction](./radial_metrics.ipynb) accross values computed from individual speckles. The [default reduction method](./radial_metrics.ipynb) is the 'mean' value for all selected speckles. For instance, if we apply this computation accross speckles at endocardium, we will have the mean values for the endocardium region. 

These are two sample speckles at Endocardium and Epicardium:

In [ ]:
sample_spk_ideal_endo = lv_ideal.get_speckles(spk_name="SAMPLE", spk_group="endo", spk_collection="SAMPLE")[0]
sample_spk_ideal_epi = lv_ideal.get_speckles(spk_name="SAMPLE", spk_group="epi", spk_collection="SAMPLE")[0]

Here is a simplified representation of how the wall thickness is calculated. Remember that the final quantity is the difference in length between the epicardium and endocardium vectors (from respective speckles to longitudinal axis reference), which may differ slightly from the visualization.

Radial metrics from the endocardium are represented by the orange lines. Green lines are epicardium radial metrics. The magenta lines represent the approximate thickness of the wall.

In [ ]:
wt = lv_ideal.plot_speckles_wall_thickness_rd(
    sample_spk_ideal_endo,
    sample_spk_ideal_epi, 
    t=0.0
)

In [ ]:
wts = lv_ideal.thickness(
    sample_spk_ideal_endo,      # selected speckles at Endocardium
    sample_spk_ideal_epi,       # selected speckles at Epicardium
    approach="radial_distance", # choice of reference LA axis
    method="mean",              # reduction method (default)
    t=0.0,                      # return value at given timestep
    recompute=True,             # forces recomputation (ignores previous computations)
    log_level=logging.ERROR     # 
    ) 

In [ ]:
print("Sample wall thickness:", wt)
print("Reduced wall thickness by internal algorithm:", wts)

### Radial Length vs Radial Distance

Considering that we use as basis for the computation a ['radial metric'](./radial_metrics.ipynb), we have two approaches to compute wall thickness: ***radial_distance*** and ***radial_length***. In addition, the user can select valid approaches for each of these radial metrics (see [docs](./radial_metrics.ipynb) for details). 


Here is a sample visualization between the two metrics. *Note that actual values might differ slightly as thickness metric is computed based on radial metric.*

In [ ]:
wt_rd = lv_typeA.plot_speckles_wall_thickness_rd(
    sample_spk_typeA_endo,
    sample_spk_typeA_epi, 
    t=0.15
)

In [ ]:
wt_rl = lv_typeA.plot_speckles_wall_thickness_rl(
    sample_spk_typeA_endo,
    sample_spk_typeA_epi, 
    t=0.15
)

In [ ]:
print("Wall thickness based on radial distance:", wt_rd)
print("Wall thickness based on radial length:", wt_rl)

## Comparing Wall Thickness metrics

In [ ]:
def plot_all_wall_thickness_metrics(lv, endo_spks, epi_spks, title=""):
        timestep = lv.timesteps()
        mask = timestep >=0.1

        rd1 = lv.radial_distance(endo_spks, 
                                approach="fixed_vector",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        rd1 = lv.radial_distance(epi_spks, 
                                approach="fixed_vector",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        wt_rd1 = lv.thickness(endo_spks, epi_spks,
                                approach="radial_distance",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        
        rd2 = lv.radial_distance(endo_spks, 
                                approach="moving_vector",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        rd2 = lv.radial_distance(epi_spks, 
                                approach="moving_vector",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        wt_rd2 = lv.thickness(endo_spks, epi_spks,
                                approach="radial_distance",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        
        
        rl1 = lv.radial_length(endo_spks, 
                                approach="fixed_centers",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        rl1 = lv.radial_length(epi_spks, 
                                approach="fixed_centers",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        wt_rl1 = lv.thickness(endo_spks, epi_spks,
                                approach="radial_length",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        
        rl2 = lv.radial_length(endo_spks, 
                                approach="moving_centers",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        rl2 = lv.radial_length(epi_spks, 
                                approach="moving_centers",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        wt_rl2 = lv.thickness(endo_spks, epi_spks,
                                approach="radial_length",
                                recompute=True, 
                                log_level=logging.ERROR)[mask]
        
        
        
        timestep = timestep[mask] - 0.1
        data = np.hstack([
                        timestep.reshape((-1,1)), 
                        wt_rd1.reshape((-1,1)), 
                        wt_rd2.reshape((-1,1)),
                        wt_rl1.reshape((-1,1)), 
                        wt_rl2.reshape((-1,1))
                        ])
        
        import pandas as pd
        
        df = pd.DataFrame(data,columns=[
                "timestep", 
                "wall_thickness_radial_distance_fixed_vector", 
                "wall_thickness_radial_distance_moving_vector", 
                "wall_thickness_radial_length_fixed_centers",
                "wall_thickness_radial_length_moving_centers",
                ])
        df.plot(x="timestep", figsize=(10,5), grid=True, title=title)

### Effect of different speckle thickness

In [ ]:
endo_thick_spks = lv_ideal.get_speckles(spk_name="THICK", spk_group="endo", spk_collection="SAMPLE")
epi_thick_spks = lv_ideal.get_speckles(spk_name="THICK", spk_group="epi", spk_collection="SAMPLE")

endo_mid_spks = lv_ideal.get_speckles(spk_name="MID", spk_group="endo", spk_collection="SAMPLE")
epi_mid_spks = lv_ideal.get_speckles(spk_name="MID", spk_group="epi", spk_collection="SAMPLE")

endo_thin_spks = lv_ideal.get_speckles(spk_name="THIN", spk_group="endo", spk_collection="SAMPLE")
epi_thin_spks = lv_ideal.get_speckles(spk_name="THIN", spk_group="epi", spk_collection="SAMPLE")


plot_all_wall_thickness_metrics(lv_ideal, endo_thick_spks, epi_thick_spks, 
                                "Ideal, using THICK speckles.")
plot_all_wall_thickness_metrics(lv_ideal, endo_mid_spks, epi_mid_spks, 
                                "Ideal, using MID speckles.")
plot_all_wall_thickness_metrics(lv_ideal, endo_thin_spks, epi_thin_spks, 
                                "Ideal, using THIN speckles.")

Same comparison, but on a typeA geometry:

In [ ]:
endo_thick_spks = lv_typeA.get_speckles(spk_name="THICK", spk_group="endo", spk_collection="SAMPLE")
epi_thick_spks = lv_typeA.get_speckles(spk_name="THICK", spk_group="epi", spk_collection="SAMPLE")

endo_mid_spks = lv_typeA.get_speckles(spk_name="MID", spk_group="endo", spk_collection="SAMPLE")
epi_mid_spks = lv_typeA.get_speckles(spk_name="MID", spk_group="epi", spk_collection="SAMPLE")

endo_thin_spks = lv_typeA.get_speckles(spk_name="THIN", spk_group="endo", spk_collection="SAMPLE")
epi_thin_spks = lv_typeA.get_speckles(spk_name="THIN", spk_group="epi", spk_collection="SAMPLE")


plot_all_wall_thickness_metrics(lv_typeA, endo_thick_spks, epi_thick_spks, 
                                "Ideal, using THICK speckles.")
plot_all_wall_thickness_metrics(lv_typeA, endo_mid_spks, epi_mid_spks, 
                                "Ideal, using MID speckles.")
plot_all_wall_thickness_metrics(lv_typeA, endo_thin_spks, epi_thin_spks, 
                                "Ideal, using THIN speckles.")

### Comparing Wall Thickness at different locations of the geometry:

TO DO.